# La modélisation : RandomForestClassifier, LogisticRegression, Linear SVC

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# 1. Charger les données
train = pd.read_csv("train.csv")

# 2. Créer la feature FamilySize
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1

# 3. Extraire le titre depuis le nom
def get_title(name):
    return name.split(",")[1].split(".")[0].strip()

train["Title"] = train["Name"].apply(get_title)

# 4. Regrouper les titres rares
rare_titles = ['Lady','Monsieur','Madame']
train["Title"] = train["Title"].replace(rare_titles, "Rare")

# 5. Remplir les valeurs manquantes
train['Age'] = train.groupby('Sex')['Age'].transform(lambda x: x.fillna(x.mean()))
train["Embarked"].fillna(train["Embarked"].mode()[0], inplace=True)

# 6. Supprimer la colonne Name (inutile)
train = train.drop(columns=["Name", "Ticket", "Cabin", "PassengerId"])

# 7. Transformer les variables catégorielles en numériques (one-hot)
train = pd.get_dummies(train, columns=["Sex", "Embarked", "Title"], drop_first=True)

# 8. Séparer les données en features X et cible y
X = train.drop("Survived", axis=1)
y = train["Survived"]

# 9. Diviser en jeu d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Liste des modèles
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'LinearSVC': LinearSVC(max_iter=10000),
}

# Évaluer chaque modèle
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"{name} accuracy: {acc:.4f}")

# Sélectionner le meilleur modèle
best_model_name = max(results, key=results.get)
best_model = models[best_model_name]
print(f"\n✅ Le meilleur modèle est : {best_model_name} avec une accuracy de {results[best_model_name]:.4f}")




import pandas as pd

# Charger les fichiers
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")  # Pour récupérer les colonnes utilisées à l'entraînement
passenger_ids = test["PassengerId"]

# ➤ Prétraitement du fichier train (même que lors de l'entraînement)
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1
train["Title"] = train["Name"].apply(lambda x: x.split(",")[1].split(".")[0].strip())
train["Title"] = train["Title"].replace(['Lady', 'Monsieur', 'Madame'], "Rare")
train["Age"] = train.groupby("Sex")["Age"].transform(lambda x: x.fillna(x.mean()))
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])
train = train.drop(columns=["Name", "Ticket", "Cabin", "PassengerId"])
train_encoded = pd.get_dummies(train, columns=["Sex", "Embarked", "Title"], drop_first=True)

# ➤ Aligner les colonnes du test avec celles du train
test_encoded = test_encoded.reindex(columns=train_encoded.drop("Survived", axis=1).columns, fill_value=0)

# ➤ Prédiction avec le meilleur modèle
predictions = best_model.predict(test_encoded)

# ➤ Création du fichier de soumission
submission = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Survived": predictions
})
submission.to_csv("submission.csv", index=False)
print("Fichier 'submission.csv' créé avec succès.")


/tmp/ipykernel_866/3905206284.py:28: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





LogisticRegression accuracy: 0.8156
RandomForest accuracy: 0.8436
LinearSVC accuracy: 0.8045

✅ Le meilleur modèle est : RandomForest avec une accuracy de 0.8436
Fichier 'submission.csv' créé avec succès.
RandomForest accuracy: 0.8436
LinearSVC accuracy: 0.8045

✅ Le meilleur modèle est : RandomForest avec une accuracy de 0.8436
Fichier 'submission.csv' créé avec succès.


# La matrice de confusion 

In [13]:
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, classification_report

# Prédire avec le meilleur modèle
y_pred_best = best_model.predict(X_test)

# Calculer la matrice de confusion
cm = confusion_matrix(y_test, y_pred_best)
labels = ["Pas survécu", "Survécu"]

# Créer une figure Plotly
fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=labels,
    y=labels,
    hoverongaps=False,
    colorscale='Blues',
    text=cm,
    texttemplate="%{text}"
))

fig.update_layout(
    title=f"Matrice de confusion - {best_model_name}",
    xaxis_title="Prédit",
    yaxis_title="Réel",
    xaxis=dict(constrain='domain'),
    yaxis=dict(constrain='domain'),
)

fig.show()

# Afficher le rapport de classification
print("\n📊 Rapport de classification :")
print(classification_report(y_test, y_pred_best, target_names=labels))



📊 Rapport de classification :
              precision    recall  f1-score   support

 Pas survécu       0.87      0.86      0.87       105
     Survécu       0.80      0.82      0.81        74

    accuracy                           0.84       179
   macro avg       0.84      0.84      0.84       179
weighted avg       0.84      0.84      0.84       179



# Modéle KNN : Regression

In [18]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Charger les données
df = pd.read_csv("train.csv")

# Garder seulement les colonnes utiles
df = df[["Sex", "Age"]]

# Supprimer les lignes où l’âge est manquant
df = df.dropna()

# Convertir la colonne 'Sex' en numérique (male=0, female=1)
        
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

# Séparer les variables
X = df[["Sex"]]
y = df["Age"]

# Séparer en train et test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer le modèle KNN Regressor avec 5 voisins
knn_reg = KNeighborsRegressor(n_neighbors=5)

# Entraîner le modèle
knn_reg.fit(X_train, y_train)

# Prédire sur le jeu de test
y_pred = knn_reg.predict(X_test)

# Évaluer les performances
mse = mean_squared_error(y_test, y_pred)
print("Erreur quadratique moyenne (MSE) :", mse)

# Exemple de prédiction : prédire l'âge d'une femme (Sex = 1)
example_female = pd.DataFrame({"Sex": [1]})
predicted_age_female = knn_reg.predict(example_female)
print("Âge prédit pour une femme :", predicted_age_female[0])

# Exemple de prédiction : prédire l'âge d'un homme (Sex = 0)
example_male = pd.DataFrame({"Sex": [0]})
predicted_age_male = knn_reg.predict(example_male)
print("Âge prédit pour un homme :", predicted_age_male[0])

# Racine de l’erreur quadratique moyenne (RMSE) : 13.691046829330224

Erreur quadratique moyenne (MSE) : 187.92174825174823
Âge prédit pour une femme : 28.2
Âge prédit pour un homme : 30.0


# Modéle Lenear SCV : Classification

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Sélectionner les colonnes pertinentes
features = ["Pclass", "Sex", "Age", "SibSp", "Embarked"]
df = pd.read_csv("train.csv")
df = df[features + ["Survived"]]


# Remplacer les âges manquants par la moyenne selon le sexe
df['Age'] = df.groupby('Sex')['Age'].transform(lambda x: x.fillna(x.mean()))
# Remplacer Embarked manquant par la valeur la plus fréquente
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)


# Encodage des variables catégorielles
import sklearn
from sklearn.preprocessing import LabelEncoder
le_sex = LabelEncoder()
df['Sex'] = le_sex.fit_transform(df['Sex'])
le_embarked = LabelEncoder()
df['Embarked'] = le_embarked.fit_transform(df['Embarked'])


# Séparer les données en features X et cible y
X = df.drop("Survived", axis=1)
y = df["Survived"]


# Diviser en jeu d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Créer et entraîner le modèle RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


# Faire des prédictions sur le test
y_pred = model.predict(X_test)


# Afficher les résultats
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

/tmp/ipykernel_826/2250359587.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)


Accuracy: 0.8100558659217877

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.87      0.84       105
           1       0.79      0.73      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179

